%md
# EML4 - Aula 05 - Processamento de Massivo de Dados
### Preparação de dados e Aprendizado de Máquina em Spark


Principais links:
* [Spark docs](https://spark.apache.org/docs/latest/)
* [Datasets Compactados](https://drive.google.com/file/d/1MdZGO9quJVxuq5wcBafT25kA2UR4C1uG/view?usp=sharing)

In [0]:
#Importações e inicialização da sessão Spark para Notebooks fora do Databricks

#import findspark, pyspark
#from pyspark.sql import SparkSession
#findspark.init()
#spark = SparkSession.builder.getOrCreate()

#Checando se sessão Spark está funcionando
spark

### Preparação

Carregando o data set. O arquivo [crimes.csv](https://www.kaggle.com/ankkur13/boston-crime-data) deve ser adicionado ao cluster e caminho colocado na célula abaixo.

Este data set possui formato csv (comma-separated values) e possui o título das colunas, mas Spark aceita diferentes valores como Parquet, ORC, JDBC, LIBSVM e outros.

In [0]:
dataset_location = 'dbfs:/FileStore/shared_uploads/naldi@ufscar.br/crime.csv'

dataset = spark.read.format('csv') \
               .option('inferSchema', True) \
               .option('header', True) \
               .option('sep', ',') \
               .load(dataset_location)

In [0]:
#Aternativamente dataset.show(?)
display(dataset)

In [0]:
dataset.printSchema()

###Transformers e Estimators

**Exemplo 1**: Tokenizer é transformer para manipulação de dados. Seu objetivo é quebrar uma String em um vetor de tokens que podem ser manipulados

In [0]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer(inputCol='OFFENSE_CODE_GROUP', outputCol='OFFENSE_CODE_TOKENIZED')
df = dataset.select('OFFENSE_CODE_GROUP').distinct()
tkn.transform(df).show()

**Exemplo 2**: StandardScaler é um estimator com o objetivo de ajustar valores contínuos de forma obter um valor médio = 0 e/ou um desvio padrão = 1

Vamos padronizar os dados da localidade dos dados, filtrando dados nulos e inconsistentes? Veja o exemplo com a latitude

StandardScaler deve receber um vetor numerico como entrada, por isso utilizamos a transformação VectorAssembler para organizar os dados em um vetor

In [0]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

df = dataset.select('Lat').filter('Lat is not null').filter('Lat > 0')
local_assembler = VectorAssembler(inputCols=["Lat"], outputCol="VetLat")
dfv = local_assembler.transform(df)
mms = StandardScaler(inputCol='VetLat', outputCol='LatStd', withStd=True, withMean=True)
mms.fit(dfv).transform(dfv).show(truncate=False)

#### Exercício 1
Outro estimator, semelhante ao StandardScaler é o MinMaxScaler, seu objetivo é transformar valores de forma a se ajustarem a um limite mínimo e máximo. Seu uso mais comum é na normalização de valores para o intervalo [0,1]

Estude as características do atributo Longitude, selecione latutide e longitude válidos e aplique o estimator MinMaxScaler

In [0]:
#faça aqui o exercício 01

#### Imputer

**Exemplo 3**: Tratando valores ausentes com Imputer, substituindo pela média

In [0]:
from pyspark.ml.feature import Imputer

df = dataset.select("Lat","Long")
imput = Imputer(inputCols=["Lat","Long"], outputCols = ["NeWLat","NeWLong"])
modelo = imput.fit(df)
df = modelo.transform(df)
df.filter('Lat is null').show() 

#### RFormula

**Exemplo 4**: Utilizando o estimator RFormula

In [0]:
from pyspark.ml.feature import RFormula

df = dataset\
  .select('OFFENSE_CODE_GROUP', 'YEAR', 'MONTH', 'HOUR', 'DISTRICT')\
  .where('DISTRICT is not null')
rf = RFormula(formula = 'OFFENSE_CODE_GROUP ~ . -DISTRICT')
rf.fit(df).transform(df).show()

**Exemplo 4**: Estruturando atributos categóricos

In [0]:
df = dataset\
  .select('OFFENSE_CODE_GROUP', 'DAY_OF_WEEK', 'HOUR')
rf = RFormula(formula = 'OFFENSE_CODE_GROUP ~ DAY_OF_WEEK + HOUR')
rf.fit(df).transform(df).show()

#### Exercício 2

Utilizando RFormula, prepare um DataFrame para uma tarefa de classificação, onde o label será `OFFENSE_CODE_GROUP`, e os atributos serão: `DISTRICT`, `DAY_OF_WEEK` e `HOUR`.

Será necessário tratar valores nulos

In [0]:
#faça aqui o exercício 02

# Classificação

Preparação

In [0]:
dataset_location = '/FileStore/tables/covtype.data'

dataset = spark.read.format('csv') \
               .option('inferSchema', True) \
               .option('header', False) \
               .option('sep', ',') \
               .load(dataset_location) \
               .withColumnRenamed('_c54', 'class')

rf = RFormula(formula = 'class ~ .')
bInput = rf.fit(dataset) \
           .transform(dataset) \
           .select('features', 'label')

bInput.show()

### LogisticRegression

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
model = lr.fit(bInput)

In [0]:
print(lr.explainParams())

In [0]:
print("accuracy: ", model.summary.accuracy)
print("precision by label: ", model.summary.precisionByLabel)
print("recall by label: ", model.summary.recallByLabel)

In [0]:
test = bInput.sample(fraction=0.0001)
model.transform(test).show()

### DecisionTree

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

train, test = bInput.randomSplit([0.7, 0.3])

dt = DecisionTreeClassifier()
model = dt.fit(train)
predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("accuracy:", accuracy)

In [0]:
print(model.toDebugString)

#### Exercício 3

* Carregue os dados do dataset `Crimes in Boston`
* Pré-processe os dados para uma tarefa de classificação. A coluna `OFFENSE_CODE_GROUP` será usada como classe e as colunas `DISTRICT`, `DAY_OF_WEEK`, `HOUR` e `SHOOTING` serão usadas como atributos.
 * Registros sem informações de `DISTRICT` não devem entrar na análise
 * Registros sem informação de `SHOOTING` devem assumir valor `N` (dica: usar `expr` e `coalesce`)
* Crie um modelo de Regressão Logística
* Avalie o modelo pela métrica accuracy

In [0]:
#faça aqui o exercício 03

# Agrupamento

Preparação

In [0]:
dataset_location = '/FileStore/tables/covtype.data'

dataset = spark.read.format('csv') \
               .option('inferSchema', True) \
               .option('header', False) \
               .option('sep', ',') \
               .load(dataset_location) \
               .withColumnRenamed('_c54', 'class')

rf = RFormula(formula = ' ~ . -class')
bInput = rf.fit(dataset) \
           .transform(dataset) \
           .select('features', 'class')

bInput.show()

#### k-means

In [0]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(7)
model = kmeans.fit(bInput)

model.clusterCenters()

In [0]:
predictions = model.transform(bInput)
predictions.show()

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)

print("silhouette: ", silhouette)

#### Exercício 4

* Carregue os dados do dataset `Crimes in Boston`
* Pré-processe os dados para uma tarefa de agrupamento. As colunas `Lat` (latitude) e `Long` (longitude) serão usadas como atributos.
 * Remover ou imputar valores nulos ou inconsistentes para ambos os atributos
* Crie um modelo de agrupamento pelo algoritmo k-means
* Avalie o modelo pelo índice silhueta

In [0]:
#faça aqui o exercício 04

# Pipeline

É possível executar todos os objetos (modelos e transformações) instanciados em um *pipeline* de aprendizado de máquina. Isso facilita a aplicação de modelos e reutilização.

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

iris = spark.read.csv("dbfs:/FileStore/shared_uploads/naldi@ufscar.br/iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

vector = VectorAssembler(inputCols=["sepal_length","sepal_width","petal_length","petal_width"],outputCol="features" )

indexer = StringIndexer(inputCol="species", outputCol="target")

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,3], featuresCol="features", labelCol="target")

pipeline = Pipeline(stages=[vector, indexer, mlp])
modelo = pipeline.fit(irisTreino)
previsao = modelo.transform(irisTeste)

previsao.select("target","features","rawprediction","probability","prediction").show(5, truncate=False)

performance = MulticlassClassificationEvaluator(labelCol="target",predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)


# Crossvalidation e Tunning

Ao desenvolver aplicações com aprendizado de máquina, é possível buscar por parametrização adequada aos dados. É possível buscar pela melhor configuração utilizando *grid search* em conjunto com um bom estimador como validação cruzada.

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

iris = spark.read.csv("dbfs:/FileStore/shared_uploads/naldi@ufscar.br/iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.8,0.2])

vector = VectorAssembler(inputCols=["sepal_length","sepal_width","petal_length","petal_width"],outputCol="features" )

indexer = StringIndexer(inputCol="species", outputCol="target")

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,3], featuresCol="features", labelCol="target")

pipeline = Pipeline(stages=[vector, indexer, mlp])

performance = MulticlassClassificationEvaluator(labelCol="target", metricName="accuracy")

grid = ParamGridBuilder().addGrid(mlp.maxIter,[10,100,1000]).addGrid(mlp.layers,[[4,4,4,3],[4,6,3]]).build()

#A validação cruzada tem seu próprio avaliador e vai subdividir os dados de treino para o processo de tunning
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=performance,numFolds=5)

#Podemos utilizar e avaliar novamente o modelos escolhido pela validação cruzada (validação da validação) usando hold out
modelo = crossval.fit(irisTreino)
previsao = modelo.transform(irisTeste)
previsao.select("rawprediction","probability","prediction").show(5, truncate=False)

performance = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)





#### Exercício 7

Escolha outro conjunto de dados e induza um modelo preditivo de classificação/regressão utilizando validação cruzada e ajuste de parâmetros. Avalie o resultado escolhido por meio de acurácia. 

Utilize sua documentação disponível em [aqui!](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html?highlight=crossvalidator)

In [0]:
#faça aqui o exercício 07